<a href="https://colab.research.google.com/github/powidla/HSE-tasks-ML-in-R/blob/main/fire_ipynb_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
library(crayon, quietly=T); # load first to avoid error relating to unloading this package
install.packages("nnet", quiet=T)      # 3 sec. multinomial logistic regression: multinom()
install.packages("MASS", quiet=T)      # 9 sec. binary logistic regression:  glm(..., family=binomial);   LDA/QDA: lda(), qda()
install.packages("tidyverse", quiet=T) # 7 sec.
install.packages("glmnet", quiet=T)    # 1 min. multinomial logistic regression: glmnet(..., family=multinomial).
install.packages("FNN", quiet=T)       # KNN Regression. Docs: https://cran.r-project.org/web/packages/FNN/FNN.pdf
install.packages('reshape2', quiet=T)  # has an easy melt() function (vs gather, stack, reshape)
devtools::install_github("mkearney/kaggler", quiet=T) # https://github.com/mkearney/kaggler
library(httr, quietly=T); library(stringr, quietly=T); library(glmnet, quietly=T); library(nnet, quietly=T); library(MASS, quietly=T); 
library(tidyverse, quietly=T); library(reshape2, quietly=T); library(RColorBrewer, quietly=T);
options(repr.plot.width=20, repr.plot.height=8)   # change plot size

also installing the dependencies ‘iterators’, ‘foreach’, ‘shape’, ‘Rcpp’, ‘RcppEigen’


also installing the dependency ‘plyr’


Installing 2 packages: openssl, jsonlite

Loaded glmnet 4.1-6

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ forcats 1.0.0
✔ readr   2.1.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ ggplot2::%+%()  masks crayon::%+%()
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ dplyr::select() masks MASS::select()
✖ tidyr::unpack() masks Matrix::unpack()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [3]:
Competition = '6feb23shop'
FileCSV = 'XY_Shop.csv'
FileZip = 'XY_Shop.zip'
# # capture.output( kaggler::kgl_auth(creds_file='kaggle.json'), file='log_auth.txt')     # log is deleted when session ends
# url = paste0('https://www.kaggle.com/api/v1/competitions/data/download/', Competition,'/', FileCSV)
# FileURL = httr::GET(url, kaggler::kgl_auth())$url   # extract full path to zipped data file
# download.file(FileURL, FileZip, quiet=T, mode="wb") # download zipped data file
# system(paste0('unzip ', FileZip), intern=T)         # oddly unzip() fails to properly unzip
# kaggler::kgl_competitions_leaderboard_view(Competition)
# timeLimit = 2*60  # seconds

In [4]:
XY = read.csv(FileCSV, na.strings="", stringsAsFactors=T)  # replace "" with NA
YCol = 'Rev'       # target column
NumericCols =  unlist(lapply(XY, is.numeric))   # find numeric columns
vX = XY[is.na(XY[YCol]), !(names(XY) %in% c(YCol))]  # testing inputs only
tXY = XY[!is.na(XY[YCol]), ]  # training inputs with output
tY = tXY[YCol]                # training response
tX = tXY[,1:(ncol(tXY)-1)]
tail(vX, 2)  # a few rows from the test set
tail(tX, 20)  # a few rows from the train set
tail(tXY, 2) # a few rows from the training set

,Adm,AdmDur,Inf.,InfDur,Prd,PrdDur,BncRt,ExtRt,PgVal,SpclDay,Mo,OS,Bsr,Rgn,TfcTp,VstTp,Wkd
,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
49999,4,357.05,0,0,27,1278.00,0.00000,0.006812,14.53818,0,11,2,2,5,2,0,0
50000,0,0.00,0,0,26,1646.42,0.00755,0.032493,0.00000,0,2,2,2,4,3,0,1


,Adm,AdmDur,Inf.,InfDur,Prd,PrdDur,BncRt,ExtRt,PgVal,SpclDay,Mo,OS,Bsr,Rgn,TfcTp,VstTp,Wkd
,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
499981,0,0.00,0,0.00,5,115.83,0.000000,0.039591,0.000000,0.0,4,2,4,1,1,0,0
499982,0,0.00,0,0.00,13,70.33,0.136441,0.120430,0.000000,0.0,8,2,2,2,8,0,0
499983,0,0.00,0,0.00,12,153.39,0.000000,0.008253,0.000000,0.0,11,2,2,2,2,1,0
499984,0,0.00,0,0.00,3,17.00,0.000000,0.058506,0.000000,0.0,12,2,2,5,3,0,0
499985,0,0.00,0,0.00,12,69.97,0.142227,0.125776,0.000000,0.3,6,1,1,2,3,0,0
499986,6,96.34,0,0.00,27,671.81,0.000000,0.020310,19.033214,0.0,2,1,1,2,7,0,0
499987,2,64.24,0,0.00,15,440.68,0.000000,0.006229,134.720599,0.0,5,1,1,1,4,1,1
499988,7,200.44,0,0.00,19,542.92,0.000000,0.002366,0.000000,0.0,6,2,2,2,8,1,1
499989,0,0.00,0,0.00,5,56.00,0.034107,0.079813,0.000000,0.0,4,2,3,1,1,0,1


,Adm,AdmDur,Inf.,InfDur,Prd,PrdDur,BncRt,ExtRt,PgVal,SpclDay,Mo,OS,Bsr,Rgn,TfcTp,VstTp,Wkd,Rev
,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
499999,0,0.00,0,0,27,1185.14,0,0.001593,0,0,5,2,2,2,3,0,1,0
500000,6,51.36,0,0,59,1898.21,0,0.003224,0,0,12,2,2,2,1,0,0,0


In [ ]:
t0 = Sys.time()

<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Pipeline**
 
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc. 
1. Why did you choose these elements? (Perhaps something in EDA or prior experience lead you to these)
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**

Были испробованы различные комбинации признаков. Рассмотрено увеличение размера обучающей выборки. Использование полной выборки не принесло никаких значительных изменений.

Введены новые признаки:



1.   log(AdmDur+InfDur+PrdDur+0.000001)
2.   SpclDay*Wkd - признак праздничного дня
3.   ((Adm+Prd)>0)*1
4.   log(PgVal/(ExtRt+0.00001)+0.000001)
5.   log(ExtRt*BncRt+0.000001)
6.   log(PgVal/(BncRt+0.00001)+0.000001)

Использование данных признаков позволило поднять точность на 0.09 по сравнению с базовым вариантом






Далее была произведена нормализация всех признаков и произведено обучение модели на нормализованном наборе данных

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful. 

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**

Были рассмотрены и испробованы следующие модели регресии:



1.   multinom из пакета nnet
1.   knn.reg из пакета FNN
1.   nnet из пакета nnet
2.   glm из пакета MASS 
2.   glmnet из пакета glmnet

Наилучший результат показал knn.reg (k Nearest Neighbor Regression) 

Далее произведен подбор гиперпараметра для knn.reg Наилучшее значение k = 3



**Запись решения **

In [5]:
XY = read.csv(FileCSV, na.strings="", stringsAsFactors=T)  # replace "" with NA
YCol = 'Rev'       # target column
NumericCols =  unlist(lapply(XY, is.numeric))   # find numeric columns
vX = XY[is.na(XY[YCol]), !(names(XY) %in% c(YCol))]  # testing inputs only
tXY = XY[!is.na(XY[YCol]), ]  # training inputs with output
tY = tXY[YCol]                # training response
tX = tXY[,1:(ncol(tXY)-1)]

tX$Type1<-log(tX$AdmDur+tX$InfDur+tX$PrdDur+0.000001)
vX$Type1<-log(vX$AdmDur+vX$InfDur+vX$PrdDur+0.000001)
tX$Type2<-tX$SpclDay*tX$Wkd
vX$Type2<-vX$SpclDay*vX$Wkd
tX$Type3<-((tX$Adm+tX$Prd)>0)*1
vX$Type3<-((vX$Adm+vX$Prd)>0)*1
tX$Type4<-log(tX$PgVal/(tX$ExtRt+0.00001)+0.000001)
vX$Type4<-log(vX$PgVal/(vX$ExtRt+0.00001)+0.000001)
tX$Type5<-log(tX$ExtRt*tX$BncRt+0.000001)
vX$Type5<-log(vX$ExtRt*vX$BncRt+0.000001)
tX$Type6<-log(tX$PgVal/(tX$BncRt+0.00001)+0.000001)
vX$Type6<-log(vX$PgVal/(vX$ExtRt+0.00001)+0.000001)

In [6]:
#load *dplyr* package
library(dplyr)
#standardize 
tX <- tX %>% mutate_each_(list(~scale(.) %>% as.vector),
 vars = c('Adm' , 'AdmDur','Inf.','InfDur','Prd','PrdDur','BncRt','ExtRt','PgVal','SpclDay','OS','Mo','Bsr','VstTp','TfcTp','Wkd','Type1','Type2','Type3','Type4','Type5','Type6'))
vX <- vX %>% mutate_each_(list(~scale(.) %>% as.vector),
 vars = c('Adm' , 'AdmDur','Inf.','InfDur','Prd','PrdDur','BncRt','ExtRt','PgVal','SpclDay','OS','Mo','Bsr','VstTp','TfcTp','Wkd','Type1','Type2','Type3','Type4','Type5','Type6'))

Warning message:
“`mutate_each_()` was deprecated in dplyr 0.7.0.
ℹ Please use `across()` instead.”
Warning message:
“`mutate_each_()` was deprecated in dplyr 0.7.0.
ℹ Please use `across()` instead.”


In [7]:
library(FNN)
m2 = knn.reg(tX, vX, tY, k=3)
t(head(pY <- setNames(data.frame(m2$pred<-(m2$pred>0.5)*1), YCol), 20))  # make predictions on the test inputs, vX

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Rev,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0


In [8]:
write.csv(cbind(id=1:nrow(pY), pY), file='MySubmission.csv', row.names=F)  # write to CSV file

In [9]:
table(tY$Rev)/nrow(tY)
table(pY$Rev)/nrow(pY)


        0         1 
0.7758667 0.2241333 


      0       1 
0.78814 0.21186 

In [ ]:
RunTime = round(difftime(Sys.time(), t0, units = "secs")[[1]])
msg = paste0('Runtime is ',RunTime,' sec. Time limit is ', timeLimit, ' sec')
cat(bold(ifelse(timeLimit<RunTime, red(paste0('Limit exceeded!!! ', msg)), green(msg))))

Runtime is 72 sec. Time limit is 120 sec

## 💡**Starter Ideas**

**Model**
1. Tune model hyperparameters, batch size, optimizer, NN layers

**Features**
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Incorporate categorical features (appropriately encoded)
  1. E.g. you could replace codes (or groups of codes) with their frequencies, which may capture the implied "distance" or rarity between category levels.
  1. If encoding ordinal features with integers, should non-equidistant values be considered?

**Training observations**
1. Try clustering methods to remove similar observations. You may also try dimension reduction methods (eg. PCA) on the transposed data matrix (if it has scaled numeric features).
1. Look for and deal with outliers or influential points in the training set
1. Deal with **imbalanced sample**: oversample smaller class, or undersample larger class, or provide observation weights or provide class weights, or seek a suitable loss function
1. Investigate distributions of features. Any missing values? Any zero values?

**Predictions**
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest misclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.

**EDA and Domain Expertise**
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Learn about the domain: how should output relate to features? How do month or weekend impact users' buying activity?
  1. User Agent [&#127910;](https://www.youtube.com/results?search_query=user+agent+browser), Google Analytics [&#127910;](https://www.youtube.com/results?search_query=google+analytics), tracking online shopping intent [&#127910;](https://www.youtube.com/results?search_query=tacking+online+shopping+intent), [📄](https://scholar.google.com/scholar?q=tracking+online+shopping+intent)